# Regression Example in TensorFlow
Predicting house prices in Boston, Massachusetts 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/regression_in_tensorflow.ipynb)

#### Load dependencies

In [27]:
import numpy as np
# boston_housing is a dataset for house price prediction available in keras itself. Just like MNIST dataset.
from tensorflow.keras.datasets import boston_housing # new!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import BatchNormalization 
# model checkpoint is used to store the model trained to a file and fetch it when needed.
from tensorflow.keras.callbacks import ModelCheckpoint # new!
import os # new!

#### Load data

In [28]:
(X_train, y_train), (X_valid, y_valid) = boston_housing.load_data()

In [29]:
# housing dataset has 13 input features. Training dataset has 404 records.
X_train.shape

(404, 13)

In [30]:
# testing dataset has 102 records.
X_valid.shape

(102, 13)

In [31]:
X_train[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [32]:
y_train[0]

15.2

#### Design neural network architecture

In [33]:
model = Sequential()

# First hidden layer:
# since the dataset is small, i have reduced the number of neurons in hidden layers
# we try with 32 neurons, input dimension is 13 because there are 13 features in input dataset. 
model.add(Dense(32, input_dim=13, activation='relu'))
model.add(BatchNormalization())

# second hidden layer:
# 16 neurons, batch normalization applied
# dropout of 20% applied
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Output Layer:
# we need only one neuron in output layer, because we need to predict house price which will be a single numeric value
# since we want numerical value, activation has to be linear.
model.add(Dense(1, activation='linear'))

In [34]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 32)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,089 (4.25 KB)

 Non-trainable params: 96 (384.00 B)

#### Configure Model

In [35]:
# since it is a regression model, cost/loss has to be `mean squared error` not `cross entropy`
model.compile(loss='mean_squared_error', optimizer='adam', )

In [36]:
# output directory where the model trained will be stored.
output_dir = 'model_output/'

In [37]:
# creating sub directory for model storing, it will be 
# model_output/regression_baseline
run_name = 'regression_baseline'
output_path = output_dir + run_name

In [38]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [39]:
# specifying the model checkpoint naming format and location.
# it will be saved in output/regression_baseline directory
# file name will be like /weights.<first_two_digits_of_the_epoch>.hdf5
# also we are specifying that weights has to be stored in that file
# weights include weights and biases.
# compared to weights biases count will be less, so in some algorithms weights includes biases also.
modelcheckpoint = ModelCheckpoint(output_path + '/weights.{epoch:02d}.weights.h5', # decimal integers
                                  save_weights_only=True)


#### Train!

In [40]:
# we are fitting the model, 
# since input data is small, we are creating small batches.
# epochs = 32
# callbacks = modelcheckpoint means, models will be stored in file.
model.fit(X_train, y_train, 
          batch_size=8, epochs=32, verbose=1, 
          validation_data=(X_valid, y_valid),
          callbacks=[modelcheckpoint]) # val loss below 50 is good, 40 great

Epoch 1/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 620.5544 - val_loss: 574.1298
Epoch 2/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - loss: 605.6833 - val_loss: 603.9249
Epoch 3/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: 572.6564 - val_loss: 589.2303
Epoch 4/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - loss: 537.5297 - val_loss: 563.6761
Epoch 5/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - loss: 543.7291 - val_loss: 519.8624
Epoch 6/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 863us/step - loss: 522.0433 - val_loss: 485.0238
Epoch 7/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - loss: 482.7520 - val_loss: 451.4459
Epoch 8/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - loss: 407.5998 - val_loss: 430.3712
Epoch 9/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - loss: 412.5242 - val_loss: 382.6028
Epoch 10/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - loss: 335.8024 - val_loss: 264.9802
Epoch 11/32
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - loss: 294.2291 - val_loss: 254.0478
Epoch 12/3

#### Performing inference

In [41]:
# we are loading weights from file created for epoch 20 and doing prediction
# because for epoch 20 validation loss is lowest
# note: It is not guranteed in next run, epoch 20 will have lowest validation loss.
model.load_weights(output_path + '/weights.20.weights.h5')

In [42]:
X_valid[42]

array([  9.32909,   0.     ,  18.1    ,   0.     ,   0.713  ,   6.185  ,
        98.7    ,   2.2616 ,  24.     , 666.     ,  20.2    , 396.9    ,
        18.13   ])

In [43]:
y_valid[42]

14.1

In [44]:
model.predict(np.reshape(X_valid[42], [1, 13]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


array([[20.659891]], dtype=float32)